In [ ]:
# default_exp models.nfm

# NFM
> A pytorch implementation of Neural Factorization Machine.

NFM seamlessly combines the linearity of FM in modelling second-order feature interactions and the non-linearity of neural network in modelling higher-order feature interactions. Conceptually, NFM is more expressive than FM since FM can be seen as a special case of NFM without hidden layers.

![Untitled](https://github.com/RecoHut-Stanzas/S021355/raw/main/images/img10.png)

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.nb_imports import *
from fastcore.test import *

## V1

In [ ]:
#export
from typing import Any, Iterable, List, Optional, Tuple, Union, Callable

import torch
from torch import nn
import torch.nn.functional as F

from recohut.models.bases.common import PointModel

In [ ]:
#export
class NFM(PointModel):

    def __init__(self, n_users, n_items, embedding_dim, batch_norm=True, dropout=0.1, num_layers=3, act_function='relu'):
        """
        Args:
            n_users : int, the number of users
            n_items : int, the number of items
            embedding_dim : int, the number of latent factor
            act_function : str, activation function for hidden layer
            num_layers : int, number of hidden layers
            batch_norm : bool, whether to normalize a batch of data
            dropout : float, dropout rate
        """
        super().__init__()

        self.num_layers = num_layers

        self.user_embedding = nn.Embedding(
            num_embeddings=n_users, embedding_dim=embedding_dim
        )
        self.item_embedding = nn.Embedding(
            num_embeddings=n_items, embedding_dim=embedding_dim
        )
        self.user_bias = nn.Embedding(n_users, 1)
        self.item_bias = nn.Embedding(n_items, 1)
        self.bias_ = nn.Parameter(torch.tensor([0.0]))

        fm_modules = []
        if batch_norm:
            fm_modules.append(nn.BatchNorm1d(embedding_dim))
        fm_modules.append(nn.Dropout(dropout))
        self.fm_layers = nn.Sequential(*fm_modules)

        mlp_modules = []
        in_dim = embedding_dim
        for _ in range(num_layers):  # dim
            out_dim = in_dim # dim
            mlp_modules.append(nn.Linear(in_dim, out_dim))
            in_dim = out_dim
            if batch_norm:
                mlp_modules.append(nn.BatchNorm1d(out_dim))
            if act_function == 'relu':
                mlp_modules.append(nn.ReLU())
            elif act_function == 'sigmoid':
                mlp_modules.append(nn.Sigmoid())
            elif act_function == 'tanh':
                mlp_modules.append(nn.Tanh())
            mlp_modules.append(nn.Dropout(dropout))
        self.deep_layers = nn.Sequential(*mlp_modules)
        predict_size = embedding_dim  # layers[-1] if layers else embedding_dim

        self.pred = nn.Linear(predict_size, 1, bias=False)

        self._init_weights()

    def _init_weights(self):
        nn.init.normal_(self.item_embedding.weight, std=0.01)
        nn.init.normal_(self.user_embedding.weight, std=0.01)
        nn.init.constant_(self.user_bias.weight, 0.0)
        nn.init.constant_(self.item_bias.weight, 0.0)

        # for deep layers
        if self.num_layers > 0:  # len(self.layers)
            for m in self.deep_layers:
                if isinstance(m, nn.Linear):
                    nn.init.xavier_normal_(m.weight)
            nn.init.xavier_normal_(self.pred.weight)
        else:
            nn.init.constant_(self.pred.weight, 1.0)

    def forward(self, users, items):
        embed_user = self.user_embedding(users)
        embed_item = self.item_embedding(items)

        fm = embed_user * embed_item
        fm = self.fm_layers(fm)

        if self.num_layers:
            fm = self.deep_layers(fm)

        fm = fm + self.user_bias(users) + self.item_bias(items) + self.bias_
        pred = self.pred(fm)

        return pred.view(-1)

In [ ]:
class Args:
    def __init__(self):
        self.data_dir = '/content/data'
        self.min_rating = 4
        self.num_negative_samples = 99
        self.min_uc = 5
        self.min_sc = 5

        self.log_dir = '/content/logs'
        self.model_dir = '/content/models'

        self.val_p = 0.2
        self.test_p = 0.2
        self.num_workers = 2
        self.normalize = False
        self.batch_size = 32
        self.seed = 42
        self.shuffle = True
        self.pin_memory = True
        self.drop_last = False
        self.split_type = 'stratified'

        self.embedding_dim = 20
        self.max_epochs = 5

args = Args()

In [ ]:
ds = ML1mDataModule(**args.__dict__)

ds.prepare_data()

In [ ]:
model = NFM(n_items=ds.data.num_items, n_users=ds.data.num_users, embedding_dim=args.embedding_dim)

pl_trainer(model, ds, max_epochs=args.max_epochs)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name           | Type       | Params
----------------------------------------------
0 | user_embedding | Embedding  | 120 K 
1 | item_embedding | Embedding  | 62.5 K
2 | user_bias      | Embedding  | 6.0 K 
3 | item_bias      | Embedding  | 3.1 K 
4 | fm_layers      | Sequential | 40    
5 | deep_layers    | Sequential | 1.4 K 
6 | pred           | Linear     | 20    
----------------------------------------------
193 K     Trainable params
0         Non-trainable params
193 K     Total params
0.775     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'Test Metrics': {'apak': tensor(0.0468),
                  'hr': tensor(0.1622),
                  'loss': tensor(321.0711),
                  'ncdg': tensor(0.0732)}}
--------------------------------------------------------------------------------


[{'Test Metrics': {'apak': tensor(0.0468),
   'hr': tensor(0.1622),
   'loss': tensor(321.0711),
   'ncdg': tensor(0.0732)}}]

## V2

> **References:-**
- X He and TS Chua, Neural Factorization Machines for Sparse Predictive Analytics, 2017.
- https://github.com/rixwew/pytorch-fm/blob/master/torchfm/model/nfm.py

In [ ]:
#export
import torch

from recohut.layers.common import FeaturesEmbedding, FeaturesLinear, MultiLayerPerceptron

In [ ]:
#exporti
class FactorizationMachine(torch.nn.Module):

    def __init__(self, reduce_sum=True):
        super().__init__()
        self.reduce_sum = reduce_sum

    def forward(self, x):
        """
        :param x: Float tensor of size ``(batch_size, num_fields, embed_dim)``
        """
        square_of_sum = torch.sum(x, dim=1) ** 2
        sum_of_square = torch.sum(x ** 2, dim=1)
        ix = square_of_sum - sum_of_square
        if self.reduce_sum:
            ix = torch.sum(ix, dim=1, keepdim=True)
        return 0.5 * ix

In [ ]:
#export
class NFMv2(torch.nn.Module):
    """
    A pytorch implementation of Neural Factorization Machine.
    Reference:
        X He and TS Chua, Neural Factorization Machines for Sparse Predictive Analytics, 2017.
    """

    def __init__(self, field_dims, embed_dim, mlp_dims, dropouts):
        super().__init__()
        self.embedding = FeaturesEmbedding(field_dims, embed_dim)
        self.linear = FeaturesLinear(field_dims)
        self.fm = torch.nn.Sequential(
            FactorizationMachine(reduce_sum=False),
            torch.nn.BatchNorm1d(embed_dim),
            torch.nn.Dropout(dropouts[0])
        )
        self.mlp = MultiLayerPerceptron(embed_dim, mlp_dims, dropouts[1])

    def forward(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        cross_term = self.fm(self.embedding(x))
        x = self.linear(x) + self.mlp(cross_term)
        return torch.sigmoid(x.squeeze(1))

In [ ]:
#hide
%reload_ext watermark
%watermark -a "Sparsh A." -m -iv -u -t -d -p recohut